In [1]:
import pandas as pd
import sqlalchemy
import requests
import datetime

In [2]:
host = 'wbs-project3-db.c5v1hsxt2ic8.eu-central-1.rds.amazonaws.com'
schema = 'gans'
user = 'admin'
password = 'Awspark2022'
port = 3306
con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

In [3]:
airports_df = pd.read_sql('airports',con=con)
airports_df

,city_id,icao


In [4]:
querystring = {"withLeg":"true","withCancelled":"true","withCodeshared":"true","withCargo":"true",
               "withPrivate":"true","withLocation":"false"}
headers = {
	"X-RapidAPI-Key": "0139f8dfbdmsh2c74d8d3164ce7ap1b3e2bjsn77505670c3ca",
	"X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
}

In [5]:
today = datetime.date.today()
tomorrow = str(today + datetime.timedelta(days=1))
    
times = [["00:00","11:59"],["12:00","23:59"]]

In [6]:
icao_list = airports_df['icao'].to_list()

In [7]:
icao_list[1:2]

[]

In [8]:
flights_dict = {'ICAO':[],'departure_airport':[],'local_time':[]}
icao_list = airports_df['icao'].to_list()

for time in times:
    
    for icao in icao_list[1:2]:
        url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{icao}/{tomorrow}T{time[0]}/{tomorrow}T{time[1]}"
        response = requests.request("GET", url, headers=headers, params=querystring)
        print(response)
        flight_json = response.json()
        
        for flight in flight_json['arrivals']:
            flights_dict['ICAO'].append(icao)
            try:
                flights_dict['departure_airport'].append(flight['departure']['airport']['icao'])
            except:
                flights_dict['departure_airport'].append('unknown')
            try:
                flights_dict['local_time'].append(flight['arrival']['scheduledTimeLocal'])
            except:
                flights_dict['local_time'].append(pd.NaT)

In [9]:
flights_df = pd.DataFrame(flights_dict)

In [10]:
flights_df['local_time'] = pd.to_datetime(flights_df['local_time'])

In [11]:
flights_df.to_sql('flight',con=con,if_exists='append',index=False)

0